In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 818.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.3 MB/s eta

In [ ]:
import numpy as np
import gradio as gr
from PIL import Image
from skimage import color, filters
from tensorflow.keras.models import load_model

# Load the fused model
model_path = "/content/drive/MyDrive/fused_model1"
weight_vgg = 2.0
fused_model = load_model(model_path)

# Classes the model predicts
classes = ['Abnormal heartbeat', 'History of MI', 'MI Patients', 'Normal']

# Function to preprocess image for the fused model
def preprocess_image(image_path):
    # Convert image to grayscale
    image = Image.open(image_path)

    # Dividing the ECG leads from 1-12 from the above image
    Leads = image.crop((150, 300, 2125, 1200))
    grayscale_image = color.rgb2gray(Leads)

    # Apply Gaussian blur
    blurred_image = filters.gaussian(grayscale_image, sigma=0.7)

    # Apply Otsu thresholding
    threshold = filters.threshold_otsu(blurred_image)
    binary_image = blurred_image < threshold

    # Resize image to match model input size
    target_size = (224, 224)
    resized_image = Image.fromarray(binary_image).resize(target_size)

    # Convert to RGB
    resized_image = resized_image.convert("RGB")

    # Convert image to numpy array and normalize
    resized_image = np.array(resized_image) / 255.0

    # Add batch dimension
    resized_image = np.expand_dims(resized_image, axis=0)

    return resized_image

# Function to make predictions
def predict(image):
    # Preprocess the image
    image = preprocess_image(image)

    # Get predictions from the fused model
    predictions = fused_model.predict(image)
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_class = classes[predicted_class_index[0]]
    confidence = np.max(predictions)

    # Format output string
    result = f"Predicted Class: {predicted_class}\nConfidence: {confidence:.2f}"
    return result

# Gradio interface setup
iface = gr.Interface(fn=predict,
                     inputs=gr.Image(type="filepath", label="Upload ECG Image"),
                     outputs=gr.Text(label="Model Predictions"),
                     title="ECG Image Classification",
                     description="Predict the type of heart condition based on ECG images using a fused neural network model.")
iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://aff3620e02a10f91b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 1s 583ms/step
